# Document-Based Stores (MongoDB)

### Task 1: Create a simple MongoDB out of this relational model

This is  a toy DB about movies and actors who played roles in these movies. This DB is consisted of  

- A "Person" table who has a unique id, and a name fields.

- Another "Movie" table that has a unique id, a title, a country where it was made, and a year when it was released.

- There is (m-n) or "many-many" relationship between these two tables (i.e basically, many actors can act in many movies, and the movie include many actors)
- Therefore, we use the "Roles" table in which we can deduct which person has acted in which movie, and what role(s) they played.

<img src="RDBSchema.png" alt="3" border="0">

#### Connect to the MongoDB server, and create a mongoDB with the name 'moviedb'

In [1]:
from pymongo import MongoClient
from random import randint
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

#we use the MongoClient to communicate with the running database instance.
myclient = MongoClient("mongodb://mongo:27017/") #Mongo URI format
mydb = myclient["moviedb"]

#### Create Person/Actor collection

In [2]:
person_coll = mydb["Person"]

#### Insert the data into the Person Table

In [3]:
personList = [
  { "id": 1, "name": "Charlie Sheen" },
  { "id": 2, "name": "Michael Douglas"},
  { "id": 3, "name": "Martin Sheen"},
  { "id": 4, "name": "Morgan Freeman"}
]

persons = person_coll.insert_many(personList)

#print list of the _id values of the inserted documents:
print(persons.inserted_ids)

[ObjectId('638864009f6adeba1b49b349'), ObjectId('638864009f6adeba1b49b34a'), ObjectId('638864009f6adeba1b49b34b'), ObjectId('638864009f6adeba1b49b34c')]


#### Creating rest of Collections ("Movies", "Roles")

In [4]:
restcols = ["Movies","Roles"]

for col in restcols:
    mydb[col]

#### Inserting data into the movie Collection

In [5]:
movies_coll = mydb["Movies"]

movieList = [
  { "id": 1, "title": "Wall Street", "country":"USA","year":1987},
  { "id": 2, "title": "The American President", "country":"USA","year":1995},
  { "id": 3, "title": "The Shawshank Redemption", "country":"USA","year":1994},
]

movies = movies_coll.insert_many(movieList)

#print list of the _id values of the inserted documents:
print(movies.inserted_ids)

[ObjectId('6388642d9f6adeba1b49b34d'), ObjectId('6388642d9f6adeba1b49b34e'), ObjectId('6388642d9f6adeba1b49b34f')]


#### Inserting data into the roles Collection

In [6]:
roles_coll = mydb["Roles"]

roleList = [
  { "personId": 1, "movieId": 1, "role":["Bud Fox"]},
  { "personId": 2, "movieId": 1, "role":["Carl Fox"]},
  { "personId": 3, "movieId": 1, "role":["Gordon Gekko"]},
  { "personId": 2, "movieId": 2, "role":["A.J. MacInerney"]},
  { "personId": 3, "movieId": 2, "role":["President Andrew Shepherd"]},
  { "personId": 4, "movieId": 3, "role":["Ellis Boyd 'Red' Redding"]}
]

roles = roles_coll.insert_many(roleList)

#print list of the _id values of the inserted documents:
print(roles.inserted_ids)

[ObjectId('6388643d9f6adeba1b49b350'), ObjectId('6388643d9f6adeba1b49b351'), ObjectId('6388643d9f6adeba1b49b352'), ObjectId('6388643d9f6adeba1b49b353'), ObjectId('6388643d9f6adeba1b49b354'), ObjectId('6388643d9f6adeba1b49b355')]


### <font color ='green'>Just for your info</font>:

#### Another Way of Modeling this M-N model in Mongo would be using the Foreign Keys 


* Movies


```[

{
	"_id": 1,
	"title":"Wall Street",
	"country":"USA",
	"year":1987,
	"persons":[1,2]
},

{
	"_id": 2,
	"title":"The American President",
	"country":"USA",
	"year":1995,
	"persons":[2]
}]
```
* Actors

```
[{
    "_id": 1,
    "name": "Charlie Sheen",
    "movies":[
    {"role": "Bud Fox", "movie_id":1}
    ]
},

{
    "_id": 2,
    "name": "Micheal Douglas",
    "movies":[
    {"role": "Gordon Geko", "movie_id":1},
    {"role": "President Andrew Shepherd", "movie_id":2}
    ]
}

] ```


#### Get all actors in your Mongo DB

In [7]:
allPersons = mydb.Person.find({}) #we can ignore the empty '{}' doc.
for person in allPersons:
    print(person)

{'_id': ObjectId('638864009f6adeba1b49b349'), 'id': 1, 'name': 'Charlie Sheen'}
{'_id': ObjectId('638864009f6adeba1b49b34a'), 'id': 2, 'name': 'Michael Douglas'}
{'_id': ObjectId('638864009f6adeba1b49b34b'), 'id': 3, 'name': 'Martin Sheen'}
{'_id': ObjectId('638864009f6adeba1b49b34c'), 'id': 4, 'name': 'Morgan Freeman'}


#### Get actors with names start with 'C' letter

In [8]:
actors = mydb.Person.find({"name": {"$regex": "^C.*"}})
for actor in actors:
    print(actor)

{'_id': ObjectId('6387e90c5c90eb67ccec4e1f'), 'id': 1, 'name': 'Charlie Sheen'}


#### Get all Movies sorted from recent to old! (get only the title and year fields)

In [9]:
movies_sorted=mydb.Movies.find({}, {
    "title": 1,
    "year": 1,
    "_id": 0
}).sort([("year",-1)])
for movie in movies_sorted:
    print(movie)

{'title': 'The American President', 'year': 1995}
{'title': 'The Shawshank Redemption', 'year': 1994}
{'title': 'Wall Street', 'year': 1987}


#### Get all Movies released in the 90s (after year (1990) and before 2000) ordered from old to recent.

In [10]:
movies = mydb.Movies.find({"$and": [{"year": {"$gt":1990}}, {"year": {"$lt":2000}}]}).sort([("year",1)])

for movie in movies:
    print(movie)

{'_id': ObjectId('6387e9275c90eb67ccec4e25'), 'id': 3, 'title': 'The Shawshank Redemption', 'country': 'USA', 'year': 1994}
{'_id': ObjectId('6387e9275c90eb67ccec4e24'), 'id': 2, 'title': 'The American President', 'country': 'USA', 'year': 1995}


#### Get Movies and Actors from your "movies" DB
* Hint : use the <code>'$lookup'</code> operator.
* The Result should be something like the following:
<code>
Charlie Sheen : Wall Street
Michael Douglas : Wall Street
Martin Sheen : Wall Street
Michael Douglas : The American President
Martin Sheen : The American President
Morgan Freeman : The Shawshank Redemption
</code>

In [11]:
results = mydb.Roles.aggregate([
    {
        "$lookup": {
           "from": "Person",
           "localField": "personId",
           "foreignField": "id",
           "as": "person"
        }
    },
    {
       "$lookup": {
           "from": "Movies",
           "localField": "movieId",
           "foreignField": "id",
           "as": "movie"
        }
    },
    {
        "$project": { 
            "person.name": 1,
            "movie.title": 1,
            "_id": 0
        }
    }
])
for result in results:
    print(result["person"][0]["name"] + ": " + result["movie"][0]["title"])

Charlie Sheen: Wall Street
Michael Douglas: Wall Street
Martin Sheen: Wall Street
Michael Douglas: The American President
Martin Sheen: The American President
Morgan Freeman: The Shawshank Redemption


#### For each Actor, get count of "Movies" he acted in.

In [13]:
agg_result = mydb.Roles.aggregate([
    {  
        "$group": {
            "_id": "$personId",
            "count": {"$sum": 1 }
        }
    },
    {
        "$sort": {
            "_id": 1
        }
    }
])

for movie_count in agg_result:
    print(movie_count)

{'_id': 1, 'count': 1}
{'_id': 2, 'count': 2}
{'_id': 3, 'count': 2}
{'_id': 4, 'count': 1}


In [14]:
agg_result = mydb.Roles.aggregate([
    {  
        "$group": {
            "_id": "$personId",
            "count": {"$sum": 1 }
        }
    },
    {
        "$sort": {
            "_id": 1
        }
    },
    {
        "$lookup": {
           "from": "Person",
           "localField": "_id",
           "foreignField": "id",
           "as": "person"
        }
    },
    {
        "$project": { 
            "person.name": 1,
            "count": 1
        }
    }
])

for movie_count in agg_result:
    print(movie_count["person"][0]["name"] + ": " + str(movie_count["count"]))

Charlie Sheen: 1
Michael Douglas: 2
Martin Sheen: 2
Morgan Freeman: 1


#### In your DB, list the movies that every Actor played

In [15]:
agg_result = mydb.Roles.aggregate([
    {
        "$lookup": {
           "from": "Person",
           "localField": "personId",
           "foreignField": "id",
           "as": "person"
         }
    },
    {
       "$lookup":
         {
           "from": "Movies",
           "localField": "movieId",
           "foreignField": "id",
           "as": "movie"
         }
    },
    {  
        "$group": {
            "_id": "$personId",
            "actorName" : { "$first": '$person.name' },
            "movies": { "$push": "$movie.title" }
        }
    },
    {
        "$project": { 
            "actorName": 1,
            "movies": 1,
            "_id": 0
        }
    }
])

for actor_movie in agg_result:
    print(actor_movie)

{'actorName': ['Charlie Sheen'], 'movies': [['Wall Street']]}
{'actorName': ['Michael Douglas'], 'movies': [['Wall Street'], ['The American President']]}
{'actorName': ['Martin Sheen'], 'movies': [['Wall Street'], ['The American President']]}
{'actorName': ['Morgan Freeman'], 'movies': [['The Shawshank Redemption']]}


#### Get the Persons/Actors who acted in "Wall Street" movie
- Hint use `$lookup` , `$match` operators in the aggregation piepeline

In [15]:
agg_result = mydb.Roles.aggregate([
    {
        "$lookup": {
           "from": "Person",
           "localField": "personId",
           "foreignField": "id",
           "as": "person"
        }
    },
    {
       "$lookup": {
           "from": "Movies",
           "localField": "movieId",
           "foreignField": "id",
           "as": "movie"
        }
    },
    {  
        "$match": {
            "movie.title": "Wall Street",
        }
    },
    {
        "$project": { 
            "person.name": 1,
            "_id": 0
        }
    }
])

for actor in agg_result:
    print(actor["person"][0]["name"])

Charlie Sheen
Michael Douglas
Martin Sheen


#### Get the Movies in which "Michael Douglas" has played a role in

In [16]:
agg_result = mydb.Roles.aggregate([
    {
        "$lookup":
         {
           "from": "Person",
           "localField": "personId",
           "foreignField": "id",
           "as": "person"
         }
    },
    {  
        "$match": {
            "person.name": "Michael Douglas",
        }
    },
    {
       "$lookup":
         {
           "from": "Movies",
           "localField": "movieId",
           "foreignField": "id",
           "as": "movie"
         }
    },
    {
        "$project": { "movie.title": 1, "_id": 0 }
    }
])

for movie in agg_result:
    print(movie["movie"][0]["title"])

Wall Street
The American President


#### Get count of "Movies" in your DB

In [16]:
mydb.Movies.count_documents({})

3

#### update the year of the 'Wall Street' movie was released in to be 2000(which is not true BTW :)
- Show that movie before and After updating it

In [17]:
mydb.Movies.update_one({"title":"Wall Street"},{"$set": {"year":2000}  })

####  Delete all the persons with names start with 'M' letter.

In [21]:
mydb.Person.delete_many({"name": {"$regex": "^M.*"}})

### Task 2: Extend your Mongo-"MovieDB" 

Imagine now that we are going to extend our DB with new movies, actors, even with new directors.

- We add <b>**"The matrix"**</b> movie which was released in <b> USA, (1999)</b>, and has a new property/field "Tagline" <b>("Welcome to the Real World")</b>.
 
- We will also add 4 new actors (Person):
    - **"Keanu Reeves"** who was born in (1964). <font color='green'>Note:</font> "born" property is also new.
    - **"Carrie-Anne Moss"** who was born in (1967).
    - **"Laurence Fishburne"** who was born in (1960).
    - **"Hugo Weaving"** who was born in (1960).
    
- Moreover, we add 2 directors (Person) :
    - **"Lilly Wachowski"**, born in (1967)
    - **"Lana Wachowski"**, born in(1965)
- For these directors specify one more label/field as ("Director"). (You can add this while inserting the director documents)
    
 
- We will also create a new <b>collection "Directed" </b> that is directed from the later 2 directors to "the Matrix" movie.

#### Add the Movie "The Matrix" with the provided data to the Movies collection

In [18]:
new_movie = mydb.Movies.insert_one({
    "id": 4,
    "title": "The Matrix",
    "country": "USA",
    "year": 1999,
    "tagline": "Welcome to the Real World"
})

#### Insert the new 4 actors to the person collection

In [19]:
#Notice, How is easy to add a new feild compared to the RDB
newActorList = [
  { "id": 5, "name": "Keanu Reeves", "born":1964 },
  { "id": 6, "name": "Carrie-Anne Moss", "born":1967},
  { "id": 7, "name": "Laurence Fishburne", "born":1960},
  { "id": 8, "name": "Hugo Weaving", "born":1960}
]

newActors = person_coll.insert_many(newActorList)

#print list of the _id values of the inserted documents:
print(newActors.inserted_ids)

[ObjectId('63886ec79f6adeba1b49b357'), ObjectId('63886ec79f6adeba1b49b358'), ObjectId('63886ec79f6adeba1b49b359'), ObjectId('63886ec79f6adeba1b49b35a')]


#### Insert the new 2 directors to the person collection

In [20]:
#Notice, How is easy to add a new feild compared to the RDB
newDirectorList = [
  { "id": 9, "name": "Lilly Wachowski", "born":1967, "label": "Director" },
  { "id": 10, "name": "Lana Wachowski", "born":1965, "label": "Director" },
]

newDirectors = person_coll.insert_many(newDirectorList)

#print list of the _id values of the inserted documents:
print(newDirectors.inserted_ids)

[ObjectId('63886ed29f6adeba1b49b35b'), ObjectId('63886ed29f6adeba1b49b35c')]


#### Create the "Directed" collection, and insert the data into it 

In [21]:
directed_coll = mydb["Directed"]

newDirectedItemsList = [
  { "id": 1, "directorId": 9, "movieId":4 },
  { "id": 2, "directorId": 10, "movieId":4 },
]

newDirectedItems = directed_coll.insert_many(newDirectedItemsList)

#print list of the _id values of the inserted documents:
print(newDirectedItems.inserted_ids)

[ObjectId('63886ef69f6adeba1b49b35d'), ObjectId('63886ef69f6adeba1b49b35e')]


#### Get only the directors from the person collection (i.e. persons marked with the label "Director")

In [22]:
directorsList = mydb.Person.find({"label": "Director"})

for director in directorsList:
    print(director)

{'_id': ObjectId('63886ed29f6adeba1b49b35b'), 'id': 9, 'name': 'Lilly Wachowski', 'born': 1967, 'label': 'Director'}
{'_id': ObjectId('63886ed29f6adeba1b49b35c'), 'id': 10, 'name': 'Lana Wachowski', 'born': 1965, 'label': 'Director'}


#### Perform a query that get persons (names, and born year) who Directed "The Matrix" movie.

In [23]:
agg_result = mydb.Directed.aggregate([
    {
       "$lookup":
         {
           "from": "Movies",
           "localField": "movieId",
           "foreignField": "id",
           "as": "movie"
         }
    },
    {  
        "$match": {
            "movie.title": "The Matrix",
        }
    },
        {
        "$lookup":
         {
           "from": "Person",
           "localField": "directorId",
           "foreignField": "id",
           "as": "person"
         }
    },
    {
        "$project": { "person.name": 1, "person.born": 1, "_id": 0 }
    }
])

for result in agg_result:
    print(result)

{'person': [{'name': 'Lilly Wachowski', 'born': 1967}]}
{'person': [{'name': 'Lana Wachowski', 'born': 1965}]}


**<center> <font color='red'>THANK YOU FOR YOUR EFFORT!</font></center>**